In [21]:
import pandas as pd

In [2]:
df = pd.read_csv("multi-turn-queries.csv")

In [4]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("OPEN_API_KEY")
openai.api_key = API_KEY
client = openai

In [ ]:
df.head()

In [9]:
questions = df.values[0].tolist()

In [10]:
questions

['Write me a sentence with 25 words ', 'Count the words']

In [11]:
model = "gpt-4o-mini"

In [18]:
def run_assistant(model, questions):
    assistant = client.beta.assistants.create(model = model, temperature = 0)
    thread = client.beta.threads.create()

    for q in questions:
        message = client.beta.threads.messages.create(thread_id = thread.id, role = "user", content = q)
        run = client.beta.threads.runs.create_and_poll(thread_id = thread.id, assistant_id=assistant.id)
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        
    for message in messages.data[::-1]:
        role = message.role
        content = message.content[0].text.value
        print(f"{role.capitalize()}: {content}\n")
        
    return messages

In [30]:
import json

def run_assistant(model, questions):
    assistant = client.beta.assistants.create(model=model, temperature=0)
    thread = client.beta.threads.create()

    for q in questions:
        client.beta.threads.messages.create(thread_id=thread.id, role="user", content=q)
        client.beta.threads.runs.create_and_poll(thread_id=thread.id, assistant_id=assistant.id)

    messages = client.beta.threads.messages.list(thread_id=thread.id)

    message_history = []
    
    for message in messages.data[::-1]:
        role = message.role
        content = message.content[0].text.value
        message_history.append({
            "role": role.capitalize(),
            "content": content
        })

    return message_history

In [31]:
messages = run_assistant(model, questions)

In [35]:
messages

[{'role': 'User', 'content': 'Write me a sentence with 25 words '},
 {'role': 'Assistant',
  'content': 'In the quiet of the evening, as the sun dipped below the horizon, the stars began to twinkle, creating a magical atmosphere filled with wonder and peace.'},
 {'role': 'User', 'content': 'Count the words'},
 {'role': 'Assistant',
  'content': "I apologize for the oversight. The sentence I provided contains 26 words. Here’s a revised sentence with exactly 25 words:\n\nAs the sun set behind the mountains, the sky transformed into a canvas of vibrant colors, inspiring everyone to pause and appreciate nature's breathtaking beauty."}]